In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg


cfg.N_CLUSTER_RATION = 2000

cfg.EDIT_DISTANCE_THRESH_LONG = 0.25
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.25
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 3

from file_io import load_urls, dump_urls
from preprocess import make_ip_url_map
from vectorize import make_vectorize
from clustering import make_ip_cluster, make_string_distance_cluster
from regex_url import url_regex_extract, url_regex_check, url_regex_publish
from regex_domain import domain_regex_extract, domain_regex_check
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
import file_io

In [2]:
urls = load_urls("../../data/VirusTotalnew/malicious/20180416-su.txt", csv = False)
urls = list(set(urls))
print len(urls)

2018-04-18 09:21:41 file_io.py [line:52] DEBUG	URLs Count:	405228


405228


In [3]:
file_io.dump_urls(urls, "../../data/EXP_VirusTotal5/suspious.csv")

2018-04-18 09:21:48 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/suspious.csv
2018-04-18 09:21:49 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_VirusTotal5/suspious.csv


In [4]:
url_domain, url_path = preprocess.make_url_split(urls)
print len(url_domain)
print len(url_path)

167803
217983


In [5]:
url_domain = [ _ for _ in url_domain if not preprocess.check_ip(_)]
print len(url_domain)

165711


In [6]:
cc_list = clustering.make_kmeans_cluster_mass(url_domain, domain=True, path=False, param=False, thresh=50000)

2018-04-18 09:22:40 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(165711, 128)
2018-04-18 09:22:40 clustering.py [line:258] DEBUG	beggin to make k-means cluster, total_data_size: 165711 n_clusters: 82
2018-04-18 09:22:40 clustering.py [line:279] DEBUG	optimal n_cluster	3
2018-04-18 09:22:50 clustering.py [line:295] DEBUG	k-means cluster done!
2018-04-18 09:22:50 clustering.py [line:323] DEBUG	Preliminary K-means clustering complete
2018-04-18 09:22:58 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(141777, 128)
2018-04-18 09:22:58 clustering.py [line:258] DEBUG	beggin to make k-means cluster, total_data_size: 141777 n_clusters: 70
2018-04-18 09:22:58 clustering.py [line:279] DEBUG	optimal n_cluster	3
2018-04-18 09:23:09 clustering.py [line:295] DEBUG	k-means cluster done!
2018-04-18 09:23:12 vectorize.py [line:81] DEBUG	vectorization complete! data shape:	(61077, 128)
2018-04-18 09:23:12 clustering.py [line:258] DEBUG	beggin to make k-means cluster

In [7]:
cc_dict = {}
for n, i in enumerate(cc_list):
    cc_dict[n] = i
file_io._dump_kmeans_hier_cluster_data(cc_dict, "../../data/EXP_VirusTotal5/cluster_hier.json")

2018-04-18 09:26:31 file_io.py [line:150] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/cluster_hier.json
2018-04-18 09:26:32 file_io.py [line:154] DEBUG	cluster has been dump	../../data/EXP_VirusTotal5/cluster_hier.json


In [8]:
cc_list = file_io._load_kmeans_hier_cluster_data("../../data/EXP_VirusTotal5/cluster_hier.json")

2018-04-18 09:26:47 file_io.py [line:133] DEBUG	K-means/hier data has been loaded	../../data/EXP_VirusTotal5/cluster_hier.json


In [9]:
clustering.make_string_distance_cluster_opt(data = cc_list,
                                            n_jobs = 8,
                                            metric = "distance",  
                                            file_path = "../../data/EXP_VirusTotal5/cluster_distance.json")

batch_index 0 0/25007
batch_index 1 0/23402
batch_index 3 0/20501
batch_index 2 0/22988
batch_index 4 0/19941
batch_index 5 0/18513
batch_index 6 0/18284
batch_index 7 0/16129
batch_index 4 100/19941
batch_index 4 200/19941
batch_index 0 100/25007
batch_index 7 100/16129
batch_index 6 100/18284
batch_index 3 100/20501
batch_index 5 100/18513
batch_index 1 100/23402
batch_index 0 200/25007
batch_index 7 200/16129
batch_index 6 200/18284
batch_index 3 200/20501
batch_index 5 200/18513
batch_index 0 300/25007
batch_index 1 200/23402
batch_index 7 300/16129
batch_index 6 300/18284
batch_index 5 300/18513
batch_index 3 300/20501
batch_index 0 400/25007
batch_index 7 400/16129
batch_index 1 300/23402
batch_index 4 2300/19941
batch_index 6 400/18284
batch_index 5 400/18513
batch_index 3 400/20501
batch_index 0 500/25007
batch_index 4 2700/19941
batch_index 7 500/16129
batch_index 2 100/22988
batch_index 6 500/18284
batch_index 1 400/23402
batch_index 5 500/18513
batch_index 4 3400/19941
batch

batch_index 5 4800/18513
batch_index 0 5700/25007
batch_index 6 5100/18284
batch_index 1 3700/23402
batch_index 7 7400/16129
batch_index 3 4500/20501
batch_index 5 4900/18513
batch_index 2 1200/22988
batch_index 7 7500/16129
batch_index 6 5200/18284
batch_index 7 7600/16129
batch_index 3 4600/20501
batch_index 1 3800/23402
batch_index 7 7700/16129
batch_index 6 5300/18284
batch_index 0 5900/25007
batch_index 5 5100/18513
batch_index 3 4700/20501
batch_index 7 7900/16129
batch_index 0 6000/25007
batch_index 6 5400/18284
batch_index 1 3900/23402
batch_index 5 5200/18513
batch_index 7 8100/16129
batch_index 0 6100/25007
batch_index 6 5500/18284
batch_index 3 4800/20501
batch_index 1 4000/23402
batch_index 5 5300/18513
batch_index 0 6200/25007
batch_index 7 8300/16129
batch_index 6 5600/18284
batch_index 3 4900/20501
batch_index 0 6300/25007
batch_index 5 5400/18513
batch_index 2 1300/22988
batch_index 1 4100/23402
batch_index 7 8600/16129
batch_index 0 6400/25007
batch_index 3 5000/20501


batch_index 0 12800/25007
batch_index 6 12500/18284
batch_index 5 11800/18513
batch_index 3 9800/20501
batch_index 0 12900/25007
batch_index 6 12600/18284
batch_index 5 11900/18513
batch_index 0 13000/25007
batch_index 6 12700/18284
batch_index 3 9900/20501
batch_index 0 13100/25007
batch_index 5 12100/18513
batch_index 1 8000/23402
batch_index 3 10000/20501
batch_index 0 13200/25007
batch_index 5 12300/18513
batch_index 6 13100/18284
batch_index 1 8100/23402
batch_index 3 10100/20501
batch_index 5 12400/18513
batch_index 6 13200/18284
batch_index 0 13400/25007
batch_index 6 13300/18284
batch_index 5 12500/18513
batch_index 3 10200/20501
batch_index 0 13500/25007
batch_index 6 13400/18284
batch_index 5 12600/18513
batch_index 6 13500/18284
batch_index 0 13600/25007
batch_index 5 12700/18513
batch_index 3 10300/20501
batch_index 1 8300/23402
batch_index 5 12800/18513
batch_index 6 13700/18284
batch_index 0 13700/25007
batch_index 3 10400/20501
batch_index 5 12900/18513
batch_index 6 138

batch_index 3 20300/20501
batch_index 3 20400/20501
batch_index 3 20500/20501
batch_index 2 3400/22988
batch_index 1 12400/23402
batch_index 1 12500/23402
batch_index 1 12600/23402
batch_index 1 12700/23402
batch_index 2 3500/22988
batch_index 1 12900/23402
batch_index 1 13000/23402
batch_index 1 13100/23402
batch_index 1 13200/23402
batch_index 2 3600/22988
batch_index 1 13500/23402
batch_index 1 13600/23402
batch_index 1 13700/23402
batch_index 1 13800/23402
batch_index 1 13900/23402
batch_index 1 14000/23402
batch_index 1 14100/23402
batch_index 1 14400/23402
batch_index 1 14500/23402
batch_index 2 3800/22988
batch_index 1 14600/23402
batch_index 1 14700/23402
batch_index 1 14800/23402
batch_index 1 14900/23402
batch_index 1 15000/23402
batch_index 1 15100/23402
batch_index 1 15200/23402
batch_index 1 15300/23402
batch_index 1 15400/23402
batch_index 1 15500/23402
batch_index 1 15600/23402
batch_index 1 15700/23402
batch_index 2 4000/22988
batch_index 1 15900/23402
batch_index 1 161

2018-04-18 09:42:48 file_io.py [line:103] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/cluster_distance.json


In [4]:
regex_list = domain_regex_extract(input_file_path = "../../data/EXP_VirusTotal5/cluster_distance.json",
                                 output_file_path = "../../data/EXP_VirusTotal5/regex_raw.txt",
                                 cluster_size_thresh = 5)

2018-04-18 09:49:30 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_VirusTotal5/cluster_distance.json
2018-04-18 09:49:30 regex_domain.py [line:297] DEBUG	total cluster num:	115348
2018-04-18 09:49:30 regex_domain.py [line:299] DEBUG	big cluster:	2739
2018-04-18 09:49:30 regex_domain.py [line:301] DEBUG	small cluster:	12190
2018-04-18 09:49:30 regex_domain.py [line:302] DEBUG	single one:	100419
2018-04-18 09:49:30 regex_domain.py [line:303] DEBUG	cluster size detail:	Counter({1: 100419, 2: 8368, 3: 2571, 4: 1251, 5: 691, 6: 491, 7: 301, 8: 240, 9: 177, 10: 152, 11: 92, 12: 84, 13: 53, 14: 51, 15: 43, 16: 43, 17: 29, 19: 26, 20: 25, 21: 20, 18: 19, 22: 18, 23: 11, 25: 10, 26: 10, 24: 9, 30: 9, 29: 8, 27: 7, 28: 6, 36: 5, 31: 4, 37: 4, 40: 4, 63: 4, 33: 3, 49: 3, 59: 3, 79: 3, 34: 2, 38: 2, 41: 2, 42: 2, 46: 2, 51: 2, 55: 2, 56: 2, 58: 2, 66: 2, 91: 2, 102: 2, 107: 2, 111: 2, 1108: 1, 32: 1, 35: 1, 39: 1, 43: 1, 44: 1, 45: 1, 48: 1, 50: 1, 64: 1, 69: 1, 70: 1, 72:

2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^edu\.zwepu\.cn$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'pw'], 1: [u'qtye1'], 2: ['\\w{1,2}88fa']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{1,2}88fa\.qtye1\.pw$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'loan'], 1: [u'cvtwkf'], 2: ['[A-Za-z]{3,4}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^[A-Za-z]{3,4}\.cvtwkf\.loan$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'pc6'], 2: ['\\w{2,3}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{2,3}\.pc6\.com$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'org'], 1: [u'dns0'], 2: ['\\d{5}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\d{5}\.dns0\.org$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'uglyas'], 2

2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^www\.[A-Za-z]{4,7}\.com\.br$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: ['\\w{4,6}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{4,6}\.15525\.www\.gtiuo\.win$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: [u'gxkugs'], 5: ['\\w{4,6}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{4,6}\.gxkugs\.15525\.www\.gtiuo\.win$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'iw3fhwsvw'], 2: ['\\w{6,7}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{6,7}\.iw3fhwsvw\.com$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: ['\\w{6}'], 5: [u'blog']})
2018-04-18 09:49:32 regex_domain

2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\d{5}\.\w{5,6}\.15525\.www\.gtiuo\.win$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'xiazaidown'], 2: ['down\\d{6,8}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^down\d{6,8}\.xiazaidown\.com$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'au'], 1: [u'com'], 2: ['[A-Za-z]{3,6}'], 3: [u'www']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^www\.[A-Za-z]{3,6}\.com\.au$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: [u'oevhoq'], 5: ['\\w{5,6}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{5,6}\.oevhoq\.15525\.www\.gtiuo\.win$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: [u'e3d47v'], 5: ['qkz4t0']})
2018-04-18 09:49:32 regex_

2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^www\.[A-Za-z]{6,7}as\.com\.br$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'iw3fhwsvw'], 2: ['\\w{6,7}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^\w{6,7}\.iw3fhwsvw\.com$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'br'], 1: [u'com'], 2: ['[A-Za-z]{3,8}'], 3: [u'www']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^www\.[A-Za-z]{3,8}\.com\.br$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: ['\\w{6}'], 5: [u'boke']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBUG	^boke\.\w{6}\.15525\.www\.gtiuo\.win$
2018-04-18 09:49:32 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'gtiuo'], 2: [u'www'], 3: [u'15525'], 4: [u'wbf3p5'], 5: ['\\w{4,6}']})
2018-04-18 09:49:32 regex_domain.py [line:283] DEBU

...

2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{5,6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'aibet789'], 2: ['\\d{4,5}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\d{4,5}\.aibet789\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'lchongfu'], 2: ['\\w{6}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'jio9xexx'], 2: ['\\w{6}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6}\.jio9xexx\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'lchongfu'], 2: ['\\w{4,6}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{4,6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'ru'], 1: [u

2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{5,6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'yygczx'], 2: [u'f'], 3: [u'56793'], 4: [u'titlemdown', u'mdown']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^(:?titlemdown|mdown)\.56793\.f\.yygczx\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'31dn8uz'], 2: ['\\w{6,7}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6,7}\.31dn8uz\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'31dn8uz'], 2: ['\\w{6,8}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6,8}\.31dn8uz\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'0roptbgg23'], 2: ['\\w{6,9}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6,9}\.0roptbgg23\.com$
2018-04-18 09:49:36 regex_domain

2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{5,6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'7ndojtv'], 2: ['\\w{6}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{6}\.7ndojtv\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'lchongfu'], 2: ['\\w{5,6}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{5,6}\.lchongfu\.com$
2018-04-18 09:49:36 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'7ndojtv'], 2: ['\\w{7,8}']})
2018-04-18 09:49:36 regex_domain.py [line:283] DEBUG	^\w{7,8}\.7ndojtv\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'n82h8b77'], 2: ['[A-Za-z]{6,7}']})
2018-04-18 09:49:37 regex_domain.py [line:283] DEBUG	^[A-Za-z]{6,7}\.n82h8b77\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'

2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'lchongfu'], 2: ['\\w{6}']})
2018-04-18 09:49:37 regex_domain.py [line:283] DEBUG	^\w{6}\.lchongfu\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'6r9j0fyq16'], 2: ['\\w{6}']})
2018-04-18 09:49:37 regex_domain.py [line:283] DEBUG	^\w{6}\.6r9j0fyq16\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'pwg5p5vd'], 2: ['\\w{6,8}']})
2018-04-18 09:49:37 regex_domain.py [line:283] DEBUG	^\w{6,8}\.pwg5p5vd\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'com'], 1: [u'd30kyq1'], 2: ['\\w{6}']})
2018-04-18 09:49:37 regex_domain.py [line:283] DEBUG	^\w{6}\.d30kyq1\.com$
2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'fghs'], 2: [u'1zjq8'], 3: [u'97344'], 4: [u'b2km5a'], 5: ['\\w{5,6}']})
2018

2018-04-18 09:49:37 regex_domain.py [line:282] DEBUG	defaultdict(<type 'list'>, {0: [u'info'], 1: [u'0-0-0-0-0-0-0-0-0-0-0-0-0-7-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-59-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-41-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-45-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-53-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-38-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-33-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-47-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-56-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-60-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-52-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-28-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-44-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-23-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0-0-0-0-0-0-46-0-0-0-0-0-0-0-0-0-0-0-0-0', u'0-0-0-0-0-0-0-0

In [5]:
domain_regex_check(input_file_path      = "../../data/EXP_VirusTotal5/regex_raw.txt", 
            test_benign_file_path    = "../../data/sangfor/safe.csv",
            test_malicious_file_path = "../../data/EXP_VirusTotal5/suspious.csv",
            result_file_path         = "../../data/EXP_VirusTotal5/regex_result.txt", 
            n_jobs = 8)

2018-04-18 09:49:45 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_raw.txt
2018-04-18 09:49:47 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv
2018-04-18 09:49:47 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_VirusTotal5/suspious.csv


batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^click4bestandsafeupdates\.(:?club|win)$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^(:?yourtrulygreatforupgrade|yourtrulygreatforupgrades)\.download$
batch index 0	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^(:?perfect-codeforfreedeveloped4you|perfectcodeforfreedeveloped4you)\.(:?info|pro|top)$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^allbetterserviceupgradeclear\.review$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 90	TP(domain) 39	^\d{1,2}\.duote\.org$
batch index 1	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 119	TP(domain) 64	^\d{5}\.k9dns\.com$
batch index 0	sample index 4	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^www\.[A-Za-z]{2,3}\.com\.pe$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^edu\.cn\.rosdd\.cn$
batch index 2	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 5	TP(d

batch index 5	sample index 5	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^pm-engineering\.com\.(:?au|com)$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 520	TP(domain) 406	^\w{3,5}\.krwrn\.cn$
batch index 1	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^ns4\.ccacqsnpnpsgpfxwulne\.us$
batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^clickforbestandprepared2upgrades\.(:?info|win|top)$
batch index 7	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{6}\.g5gfmg0uw2\.com$
batch index 2	sample index 12	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 8	^sec2\d{3}doms\.co\.cc$
batch index 0	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 6	^\w{1,4}5v\.dmrxr\.cn$
batch index 4	sample index 8	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^goodupdatesyouwilluse\.top$
batch index 5	sample index 6	FP(url) 0	FP(domain) 0	TP(url) 130	TP(domain) 4	^(:?os|rp)\.(:?downloadster2cdn|downloadastrocdn)\.com$
batch index 6	sa

batch index 3	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^www\.themainplacesetnowforcontentsfree\.(:?win|bid)$
batch index 7	sample index 9	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^\d{2,3}\.\d{1,3}\.81\.118\.adsl-pool\.sx\.cn$
batch index 2	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 37	TP(domain) 37	^\w{6,8}\.p1nsu7zb8u\.com$
batch index 4	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{6}\.5bnoww26v1\.com$
batch index 0	sample index 24	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^paypal\.co\.uk\.\d{4}\.(:?secure-191|secure-911)\.mx$
batch index 6	sample index 12	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^getupdatesfreeforfunusefree\.top$
batch index 5	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 6	^\w{2}\.bkill\.com$
batch index 1	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^clickforbestandsafeupgrading\.(:?info|pw)$
batch index 3	sample index 19	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^\w{5

batch index 3	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^allthebesttoupgrades4you\.date$
batch index 2	sample index 27	FP(url) 4	FP(domain) 4	TP(url) 27	TP(domain) 24	^dx\d{2,3}\.downyouxi\.com$
batch index 1	sample index 29	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^onlineupgradefreeforsystemsetpcandmac\.download$
batch index 4	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^ftt\.\w{5,6}\.15525\.www\.gtiuo\.win$
batch index 7	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^j77\d{4}\.myjino\.ru$
batch index 0	sample index 31	FP(url) 93	FP(domain) 75	TP(url) 560	TP(domain) 293	^www\.[A-Za-z]{3,11}\.com\.br$
batch index 6	sample index 19	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^edu\.cn\.hgidck\.cn$
batch index 5	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^4or8ge\.26030\.(:?43684|4w5rog)\.q6edq9\.ow1mis\.bxnf0b\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 3	sample index 26	FP(url) 0	FP(do

...

batch index 5	sample index 99	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^www\.(:?pushingnewforcontentfree|pushingnewforcontentsset)\.download$
batch index 2	sample index 106	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^gov\.cn\.gnxxo\.cn$
batch index 3	sample index 104	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^(:?greatinsthatyouwilllovealways|greatinsthatyouwilllovetoday|greatinsthatyouwilllovenow)\.(:?info|bid|top)$
batch index 1	sample index 108	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^gatheringallusefullsoftsforyour-machinethismonth\.review$
batch index 4	sample index 102	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^clicksafe2upgrading\.pw$
batch index 7	sample index 96	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?everythingyoucanyoucanhavenowfree|everythingyoucanyoucanhavenowalways)\.(:?info|bid)$
batch index 6	sample index 98	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 6	^[A-Za-z]{1,4}an[A-Za-z]{0,4}\.000webhostapp\.com$
batch index 0	sample index 109	FP(u

batch index 2	sample index 114	FP(url) 0	FP(domain) 0	TP(url) 5	TP(domain) 5	^(:?clickforbestandsafeupgrading|clickforbestandsafe2updating)\.(:?pw|win|bid)$
batch index 5	sample index 107	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 11	^\d{5}\.hnkangermei\.com$
batch index 3	sample index 112	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 2	^\w{6}\.75033\.78124\.wnxvmn\.a4kcx0\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 4	sample index 110	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^finishedupdatefreeforsystemsmacandpc\.bid$
batch index 1	sample index 116	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^soft\.\w{5,6}\.oa69vt\.ytbyzp\.dllm3\.www\.q3537\.download$
batch index 6	sample index 106	FP(url) 0	FP(domain) 0	TP(url) 117	TP(domain) 13	^\w{4,6}\.\w{4,6}\.5clqsk\.0nfcb7\.6wqwcw\.dn7tn9\.ytbyzp\.dllm3\.www\.q3537\.download$
batch index 0	sample index 117	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^\d{2,3}\.222593\.com$
batch index 7	sample index 104	FP(url) 0	

batch index 5	sample index 115	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^yourgoodcentalforcontentnew\.(:?download|date)$
batch index 7	sample index 111	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^(:?clickforbestandreadysystems|clickforbestandreadysystem)\.(:?download|top|pro)$
batch index 0	sample index 124	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^groupsofservice2contents\.review$
batch index 2	sample index 122	FP(url) 0	FP(domain) 0	TP(url) 57	TP(domain) 29	^recover[^\.]{4,10}\.000webhostapp\.com$
batch index 4	sample index 118	FP(url) 0	FP(domain) 0	TP(url) 46	TP(domain) 26	^ame\w{5}\.beget\.tech$
batch index 3	sample index 120	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^smallandprecise2upgradesnew\.(:?club|date)$
batch index 1	sample index 124	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 6	^[^\.]{6,8}paypal[^\.]{0,2}\.myjino\.ru$
batch index 6	sample index 114	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^yourpointofforcontentingprepared\.date$
batch index 5	sample 

batch index 3	sample index 127	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebiggestofferupdate\.download$
batch index 4	sample index 125	FP(url) 0	FP(domain) 0	TP(url) 1060	TP(domain) 298	^\w{5,6}\.\w{4,6}\.\w{5,6}\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 6	sample index 121	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?yourperfectbuddyforcontentsafeall|yourperfectbuddyforcontentsafenow)\.date$
batch index 5	sample index 123	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?clickforbestandsafeupdatespcandmac|clickforbestandsafeupdatingpcandmac)\.club$
batch index 7	sample index 119	FP(url) 0	FP(domain) 0	TP(url) 65	TP(domain) 63	^blog\.\w{6}\.15525\.www\.gtiuo\.win$
batch index 3	sample index 128	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^pp-[A-Za-z]{3,4}in[^\.]{5,6}\.updater-service\.com$
batch index 4	sample index 126	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?getallyouneedforeveryonetohaveitawesome|getallyouneedforeveryonetohaveit)\.bi

2018-04-18 09:56:15 file_io.py [line:249] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/regex_result.txt
2018-04-18 09:56:16 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_VirusTotal5/regex_result.txt


In [2]:
dfo = regex_domain.domain_regex_publish("../../data/EXP_VirusTotal5/regex_result.txt",
                                  "../../data/EXP_VirusTotal5/regex_publish.txt",
                                  publish_score_thresh = 0.4,
                                  publish_ratio = 20,
                                  publish_fp_thresh = 500,
                                  publish_tp_thresh = 1)
df = dfo.loc[dfo.fp>0]
df

2018-04-18 15:07:03 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_VirusTotal5/regex_result.txt
2018-04-18 15:07:04 regex_domain.py [line:434] DEBUG	regular expression publish	978
2018-04-18 15:07:04 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-18 15:07:04 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal5/regex_publish.txt


,fp,regex,tp,score,ratio
37,2,"^[A-Za-z]{4,10}\.000webhostapp\.com$",212,0.615385,0.009481
60,2,"^\w{3,11}\.000webhostapp\.com$",429,0.607143,0.004685
108,3,"^[^\.]{3,17}\.000webhostapp\.com$",1208,0.451613,0.002492
118,25,^\d{5}\.url\.9xiazaiqi\.com$,19,0.761905,1.315623
172,41,^\d{5}\.url\.246546\.com$,15,0.722222,2.732179
243,149,^\d{5}\.url\.222bz\.com$,23,0.705882,6.475880
287,4,"^dx\d{2,3}\.downyouxi\.com$",27,0.833333,0.148464
460,3,"^[A-Za-z]{3,4}\.cvtwkf\.loan$",5,0.714286,0.600798
766,196,^down050\d{5}\.xdown4\.com$,16,0.772727,12.242973
793,1,^app-1\d{9}\.000webhostapp\.com$,22,0.709677,0.045888


In [46]:
file_io._dump_regex_list(list(df.regex), "../../data/EXP_VirusTotal5/regex_publish.txt")

2018-04-18 12:38:20 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-18 12:38:20 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal5/regex_publish.txt


In [47]:
predict_malicious, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/safe.csv",
                      regex_file_path = "../../data/EXP_VirusTotal5/regex_publish.txt")

2018-04-18 12:38:22 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-18 12:38:23 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv


batch index 0	sample index 0	hit domain 2	^[A-Za-z]{4,10}\.000webhostapp\.com$
batch index 1	sample index 0	hit domain 2	^\w{3,11}\.000webhostapp\.com$
batch index 2	sample index 0	hit domain 3	^[^\.]{3,17}\.000webhostapp\.com$
batch index 3	sample index 0	hit domain 19	^\d{5}\.url\.9xiazaiqi\.com$
batch index 4	sample index 0	hit domain 34	^\d{5}\.url\.246546\.com$
batch index 5	sample index 0	hit domain 90	^\d{5}\.url\.222bz\.com$
batch index 6	sample index 0	hit domain 4	^dx\d{2,3}\.downyouxi\.com$
batch index 7	sample index 0	hit domain 3	^[A-Za-z]{3,4}\.cvtwkf\.loan$
batch index 7	sample index 1	hit domain 158	^down050\d{5}\.xdown4\.com$
batch index 7	sample index 2	hit domain 1	^app-1\d{9}\.000webhostapp\.com$
batch index 7	sample index 3	hit domain 21	^\w{2,3}\.mqego\.com$


In [48]:
len(predict_malicious)

605

In [49]:
len(set(predict_malicious))

600

In [50]:
set(predict_malicious)

{'http://14614.url.222bz.com/down/%E6%88%91%E7%9A%84%E4%B8%96%E7%95%8C%E7%94%B5%E8%84%91%E7%89%88@804_605.exe',
 'http://15778.url.222bz.com/xiaz/%E7%94%B5%E8%BD%A6%E4%B9%8B%E7%8B%BCR%E6%9C%80%E7%BB%88%E5%AE%8C%E7%BE%8E%E5%85%B8%E8%97%8F%E7%89%88%E6%B1%89%E5%8C%96+%E8%A7%A3%E7%A0%81%E5%8E%BB%E7%A0%81+CG%E5%AD%98%E6%A1%A3+MOD+%E7%89%B9%E5%85%B8+%E6%94%BB%E7%95%A5@486_208638.exe',
 'http://18371.url.222bz.com/xiaz/CSOL2%E5%8D%95%E6%9C%BA%E7%89%88Aim%20vs%20Dim%E4%B8%8B%E8%BD%BD@547_2.exe',
 'http://18516.url.9xiazaiqi.com/down/GoogleChromeXP%3F%3F49.0.2623.112XP%3F%3F%3F%3F%3F%B7%DA@158_71467.exe',
 'http://18516.url.9xiazaiqi.com/down/GoogleChromeXP49.0.2623.112XP%3F%3F%3F%B7%DA@158_71467.exe',
 'http://18540.url.9xiazaiqi.com/xiaz/%3F%3F%3F%3F%3F%3Fv4.0.1.0%3F%3F%3F%3F%3F%3F%3F@136_30158.exe',
 'http://18540.url.9xiazaiqi.com/xiaz/%3F%3Fv4.0.1.0%3F%3F@136_30158.exe',
 'http://18565.url.9xiazaiqi.com/xiaz/%3F%3F%3F%3F%3F%3F%3F%3F%3F%3F%3F@34_329811.exe',
 'http://18565.url.9xiazaiqi.com

# test on sangfor 0410-0416 safe url

In [3]:
predict_malicious, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = "../../data/sangfor/safe0416.csv",
                      regex_file_path = "../../data/EXP_VirusTotal5/regex_publish.txt")

2018-04-18 15:07:21 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_publish.txt
2018-04-18 15:07:22 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe0416.csv


batch index 0	sample index 0	hit domain 0	^click4bestandsafeupdates\.(:?club|win)$
batch index 0	sample index 1	hit domain 0	^(:?yourtrulygreatforupgrade|yourtrulygreatforupgrades)\.download$
batch index 0	sample index 2	hit domain 0	^(:?perfect-codeforfreedeveloped4you|perfectcodeforfreedeveloped4you)\.(:?info|pro|top)$
batch index 1	sample index 0	hit domain 0	^allbetterserviceupgradeclear\.review$
batch index 0	sample index 3	hit domain 0	^\d{1,2}\.duote\.org$
batch index 0	sample index 4	hit domain 0	^www\.[A-Za-z]{2,3}\.com\.pe$
batch index 1	sample index 1	hit domain 0	^\d{5}\.k9dns\.com$
batch index 0	sample index 5	hit domain 0	^\d{3}\.555169\.com$
batch index 1	sample index 2	hit domain 0	^edu\.cn\.rosdd\.cn$
batch index 0	sample index 6	hit domain 0	^gov\.cn\.tqrth\.cn$
batch index 1	sample index 3	hit domain 0	^thecentersofttoupgrade\.(:?bid|trade)$
batch index 2	sample index 0	hit domain 0	^edu\.cn\.byeoo\.cn$
batch index 1	sample index 4	hit domain 0	^(:?getyourupdatesthat

batch index 5	sample index 4	hit domain 0	^pm-engineering\.com\.(:?au|com)$
batch index 4	sample index 9	hit domain 1	^wt\d{2,3}\.downyouxi\.com$
batch index 6	sample index 0	hit domain 0	^saisoncard-jp\.loan$
batch index 2	sample index 15	hit domain 0	^newupdates4winandmacheretoday\.review$
batch index 5	sample index 5	hit domain 0	^(:?os|rp)\.(:?downloadster2cdn|downloadastrocdn)\.com$
batch index 4	sample index 10	hit domain 0	^goodupdatesyouwilluse\.top$
batch index 3	sample index 13	hit domain 0	^www\.(:?thereadycentercontentingplacego|readycenter2contentingplace)\.(:?date|trade)$
batch index 1	sample index 23	hit domain 0	^(:?2newfreeforpcandmacsystensupdates|2nowfreeforpcandmacsystensupdates)\.(:?win|top)$
batch index 6	sample index 1	hit domain 0	^[A-Za-z]{6,7}\.blogspot\.com$
batch index 0	sample index 26	hit domain 0	^perfectservicetoupdating\.(:?club|date)$
batch index 2	sample index 16	hit domain 0	^\w{6,8}\.p1nsu7zb8u\.com$
batch index 1	sample index 24	hit domain 0	^[A-Za

batch index 0	sample index 36	hit domain 0	^j2cekf\.\w{6}\.v5tncp\.qaupgg\.a6jwhn\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 5	sample index 15	hit domain 0	^ask24updateultimateformacandpc\.(:?win|top)$
batch index 7	sample index 6	hit domain 0	^o29y2z\.\w{4,6}\.sortlh\.15525\.www\.gtiuo\.win$
batch index 4	sample index 21	hit domain 0	^\d{3}\.(:?001782|001781)\.com$
batch index 3	sample index 22	hit domain 0	^\w{6,8}\.gf1i2zwyzy\.com$
batch index 1	sample index 33	hit domain 0	^com-------mobile---read---new--terms--\d{9}\.peraltek\.com$
batch index 2	sample index 25	hit domain 0	^\w{6}\.7dtuwd\.e2lluh\.49nlpf\.1bw7f0\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 6	sample index 12	hit domain 0	^\w{4,6}\.qcd60c\.15525\.www\.gtiuo\.win$
batch index 0	sample index 37	hit domain 0	^42gnn4\.\w{6}\.4xwxiy\.0cg2zp\.17393\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 4	sample index 22	hit domain 0	^\w{6}\.ns0w4k\.36298\.yf815v\.5jso2o

batch index 1	sample index 43	hit domain 0	^[A-Za-z]{4,5}201\d{1,3}\.000webhostapp\.com$
batch index 3	sample index 32	hit domain 0	^(:?perfect-centerfornewsoftwarepcandmac|perfect-center4newsoftwarepcandmac)\.pw$
batch index 7	sample index 17	hit domain 0	^\w{2}\.sahajcare\.org$
batch index 5	sample index 25	hit domain 0	^gov\.cn\.adzow\.cn$
batch index 2	sample index 35	hit domain 0	^nowsetup\.thegreatcentralsoft2contents\.download$
batch index 4	sample index 33	hit domain 0	^(:?getsoftsthatyouwillinstallrightawayamazing|getsoftsthatyouwillinstallrightaway|getsoftsthatyouwillinstallrightawaygreat)\.(:?top|pw)$
batch index 0	sample index 48	hit domain 0	^14\d{2,11}\.xiazaidown\.com$
batch index 7	sample index 18	hit domain 0	^\w{5,6}\.n9g001\.15525\.www\.gtiuo\.win$
batch index 3	sample index 33	hit domain 0	^centerallgreatforupgrading\.date$
batch index 6	sample index 23	hit domain 0	^os\.baixakialtcdn\.com$
batch index 7	sample index 19	hit domain 0	^ready4live\.gatheringallusefulls

...

batch index 0	sample index 91	hit domain 0	^(:?goodgreatbestupdatesyoucanfind|goodgreatbestupdatesyoucanfindnow)\.top$
batch index 4	sample index 73	hit domain 0	^\w{6,8}\.22qqmb\.com$
batch index 3	sample index 75	hit domain 0	^appleid\.apple\.net\.cgi-key\.confirm\.eu\.web\.cmd\.myapple\.webscmd\.apple\.user\d{1}\.app-v0\.com$
batch index 5	sample index 67	hit domain 0	^(:?pressandclick2updates|pressandclicktoupdates)\.pro$
batch index 7	sample index 60	hit domain 0	^click4safeupdates\.(:?win|top)$
batch index 1	sample index 87	hit domain 0	^[A-Za-z]{5,7}\.bezoinprmotionvn\.info$
batch index 6	sample index 65	hit domain 0	^[A-Za-z]{6,7}\.n82h8b77\.com$
batch index 2	sample index 79	hit domain 0	^www\.your-readysystemsforcontentall\.download$
batch index 0	sample index 92	hit domain 0	^clickforbest2updates4u\.(:?bid|pw)$
batch index 3	sample index 76	hit domain 0	^setandperfectforyoursystemsvideo\.(:?top|pw)$
batch index 4	sample index 74	hit domain 0	^nowcheckserviceformacandpc\.down

batch index 7	sample index 70	hit domain 0	^www\.\w{5,6}\.\w{3,6}\.\w{3,6}\.46utso\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 0	sample index 101	hit domain 1	^[^\.]{3,17}\.000webhostapp\.com$
batch index 6	sample index 77	hit domain 0	^www\.\w{4,9}\.000webhostapp\.com$
batch index 5	sample index 77	hit domain 0	^\w{5,6}\.w3h0dm\.15525\.www\.gtiuo\.win$
batch index 1	sample index 97	hit domain 0	^gov\.cn\.jtjno\.cn$
batch index 4	sample index 85	hit domain 0	^thebestsafeandready-updates\.(:?top|pw)$
batch index 3	sample index 86	hit domain 0	^\w{6,7}\.fkmz1vn3\.com$
batch index 0	sample index 102	hit domain 0	^allthebest4upgrades4u\.bid$
batch index 6	sample index 78	hit domain 0	^adstofindgreatupdatesformactoday\.top$
batch index 5	sample index 78	hit domain 0	^\w{5,8}\.cxvtwocr\.cn$
batch index 7	sample index 71	hit domain 0	^(:?tofindgoodcontent|tofindgood2content)\.(:?date|bid)$
batch index 2	sample index 90	hit domain 0	^www\.(:?yourmainsource

batch index 7	sample index 81	hit domain 0	^theultimateandsafe2updates\.(:?bid|site)$
batch index 4	sample index 95	hit domain 0	^clicksafe2upgrading\.pw$
batch index 2	sample index 99	hit domain 0	^www\.yourmainsourceforcontentingall\.date$
batch index 0	sample index 113	hit domain 0	^\w{5,6}\.www\.pea4b\.download$
batch index 3	sample index 96	hit domain 0	^60447\.5948\.(:?nkvkla|knfgae)\.466\.0q35fr\.oa69vt\.ytbyzp\.dllm3\.www\.q3537\.download$
batch index 5	sample index 89	hit domain 0	^(:?clickforbestandsafe247upgradespcandmac|clickforbestandsafe247upgradingpcandmac)\.download$
batch index 1	sample index 109	hit domain 0	^www\.thelinkersgoodfree2contenting\.(:?date|bid)$
batch index 6	sample index 89	hit domain 0	^5u5ml1\.a24zij\.h0nn0a\.4qvcqb\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 7	sample index 82	hit domain 0	^\w{4,6}\.15525\.www\.gtiuo\.win$
batch index 4	sample index 96	hit domain 0	^gov\.cn\.evyad\.cn$
batch index 2	sample index 100	hit domain 0

batch index 1	sample index 120	hit domain 0	^dl\d{2}\.setcanadaall\.info$
batch index 2	sample index 110	hit domain 0	^softreadynow\.(:?hugetraffic-4allkindoforperatingsystemsthisweek|hugetrafficforallkindoforperatingsystemsthisweek|hugetraffic4allkindoforperatingsystemsthisweek)\.review$
batch index 5	sample index 99	hit domain 0	^\w{4,5}\.on\.nimp\.org$
batch index 6	sample index 100	hit domain 0	^ww25\.\w{2,5}\.3ddown\.com$
batch index 2	sample index 111	hit domain 0	^theforeverlastingforupgrading\.(:?club|bid)$
batch index 3	sample index 106	hit domain 0	^\d{4,5}\.url\.7wkw\.com$
batch index 4	sample index 107	hit domain 0	^(:?forultimateandbest4updatespc|forultimateandbest4updatingpc)\.(:?info|pro)$
batch index 1	sample index 121	hit domain 0	^\w{5,6}\.nb\.host-domain-lookup\.com$
batch index 7	sample index 93	hit domain 0	^theinstallpreparedupgradefree\.(:?win|bid)$
batch index 5	sample index 100	hit domain 0	^ready4newsoftsysforpcandmac\.download$
batch index 3	sample index 107	

batch index 4	sample index 118	hit domain 0	^gov\.whlxv\.cn$
batch index 5	sample index 111	hit domain 0	^[A-Za-z]{6}\.best\.lt\.ua$
batch index 3	sample index 118	hit domain 0	^service[^\.]{5,8}\.000webhostapp\.com$
batch index 5	sample index 112	hit domain 0	^fabolousuopdatesthatyouwilluse\.(:?info|top)$
batch index 6	sample index 112	hit domain 0	^(:?click2nowsafe4upgrades|click2nowsafe2upgrades|click2nowsafe2upgrade)\.win$
batch index 7	sample index 104	hit domain 0	^secure[^\.]{8,10}\.000webhostapp\.com$
batch index 4	sample index 119	hit domain 0	^ms7cq4\.di50qa\.0oxb6q\.26k4g1\.68957\.i5lyxv\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 3	sample index 119	hit domain 0	^edu\.cn\.fhlwr\.cn$
batch index 5	sample index 113	hit domain 0	^gov\.cn\.pbkcxp\.cn$
batch index 7	sample index 105	hit domain 0	^abelohost-205\.\d{3}\.(:?145|207)\.185\.dedicated-ip\.abelons\.com$
batch index 6	sample index 113	hit domain 0	^\w{5,6}\.0chhih\.19542\.15525\.www\.gtiuo\.win$
b

In [10]:
for i in predict_dict_detail.keys():
    print i,"\t",len(predict_dict_detail[i])

^bd\d{2}\.52lishi\.com$ 	1
^\w{2,3}\.mqego\.com$ 	10
^\d{5}\.url\.246546\.com$ 	7
^wt\d{2,3}\.downyouxi\.com$ 	1
^dx\d{2,3}\.downyouxi\.com$ 	3
^[^\.]{3,17}\.000webhostapp\.com$ 	1
^\d{5}\.url\.222bz\.com$ 	25


In [12]:
len(predict_malicious)

86

In [16]:
for i in predict_malicious:
    print i

http://airsick-curvature.000webhostapp.com/model.html
http://20061.url.246546.com/xiaz/%E5%90%8C%E6%B5%8E%E9%AB%98%E7%AD%89%E6%95%B0%E5%AD%A6%E7%AC%AC%E4%B8%83%E7%89%88%E4%B8%8B%E5%86%8C%E4%B9%A0%E9%A2%98%E5%85%A8%E8%A7%A3%E6%8C%87%E5%8D%97,%E5%90%8C%E6%B5%8E%E9%AB%98%E6%95%B0%E7%AC%AC%E4%B8%83%E7%89%88%E7%99%BE%E5%BA%A6%E4%BA%91%E4%B8%8B%E8%BD%BD%E8%B5%84%E6%BA%90-%E7%BD%91%E7%9B%98007@554_2.exe
http://20052.url.246546.com/xiaz/%E6%9E%97%E4%BF%8A%E6%9D%B0-%E9%86%89%E8%B5%A4%E5%A3%81.mp3%E6%9E%97%E4%BF%8A%E6%9D%B0_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007%40554_2.ex
http://20057.url.246546.com/xiaz/%E5%90%8C%E6%B5%8E%E9%AB%98%E7%AD%89%E6%95%B0%E5%AD%A6%E7%AC%AC%E4%B8%83%E7%89%88%E4%B8%8B%E5%86%8C%E4%B9%A0%E9%A2%98%E5%85%A8%E8%A7%A3%E6%8C%87%E5%8D%97,%E5%90%8C%E6%B5%8E%E9%AB%98%E6%95%B0%E7%AC%AC%E4%B8%83%E7%89%88%E7%99%BE%E5%BA%A6%E4%BA%91%E4%B8%8B%E8%BD%BD%E8%B5%84%E6%BA%90-%E7%BD%91%E7%9B%98007@554_2.exe
http://19743.url.246546.com/xiaz/17zkkj_zzfxt_013,17%E5%B9%B4%E6